### Setup

1. Create Bing Grounding Connection in project
2. Update the connection to env: BING_CONNECTION_NAME

### Connect to Foundry project

In [ ]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

# Use the connection string to connect to your Foundry project
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string, credential=DefaultAzureCredential()
    )
except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise



In [ ]:

from azure.ai.projects.models import BingGroundingTool

bing_connection = project.connections.get(connection_name=os.environ["BING_CONNECTION_NAME"])
conn_id = bing_connection.id
# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

### Create a simple agent

In [ ]:
## need to login using the command azd auth login
agent = project.agents.create_agent(
        model=os.getenv("CHAT_MODEL"),
        name="BingAgent",
        instructions="You are helpful AI assistant.",
        tools=bing.definitions,
        tool_resources=bing.resources
    )
print(f"Created agent, agent ID: {agent.id}")

### Create a thread for conversation presistance

In [ ]:
# Create a thread
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

In [ ]:
def sendMessage(msg):
    # Get messages from the thread
    message = project.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=msg
        )
    print(f"Created message, ID: {message.id}")

    run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            print(f"Run failed: {run.last_error}")
            raise Exception(f"Run failed: {run.last_error}")

    messages = project.agents.list_messages(thread_id=thread.id, order="asc", after=message.id)
    for msg in messages.data:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")


### Get the response

In [ ]:
sendMessage("What is the latest news in US?")